In [409]:
#我要翻頁!!!!!!   by 更改url
def pttcrawler(url,pages):
    response = rq.get(url,cookies={'over18': '1'})
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'lxml')
    
    this_year = datetime.datetime.now().year
    
    articles = []
    board=soup.find("a",class_= "board")["href"]
    board = re.search('bbs\/(.*)\/', board)[1]
    for s in soup.find_all("div", class_= "r-ent"):
        if (s.find('a')) and (response.status_code == 200):
            #requests.get() 的結果是 request.Response 物件, 我們可以先透過該物件的 statu_code 屬性取得 server 回覆的狀態碼
            #(例如 200 表示正常, 404 表示找不到網頁等), 若狀態碼為 200, 代表正常回應
            date=s.find("div",class_= "date").string
            date=str(this_year)+'/'+date if int(date[0:2])<=2 else str(this_year-1)+'/'+date
            date=date.replace(" ","")
            name=s.find('a').string
            push=s.find("div",class_= "nrec").string
            try:
                push=100 if push=='爆' else push
                push=-100 if push=='XX' else push 
                push=int(push.replace("X","-"))*10 if push.find("X")==0 else push
            except:
                continue
            author=s.find("div",class_= "author").string
            url=s.find("div",class_= "title").a["href"]
            articles.append({'board': board,'title': name,'push': push,'date': date,'author': author,'url': "https://www.ptt.cc"+url}) 
            
    soup2="https://www.ptt.cc"+soup.find_all(class_="btn wide")[1]["href"]#第二個([1])class是"btn wide"之下的href值
    begin=soup2.find("index")+5
    end=soup2.find(".html")
#     try:
#         int(soup2[-9:-5])
#     except:
#         try:
#             int(soup2[-8:-5])
#         except:
#             last_num=-7
#         else:
#             last_num=-8
#     else:
#         last_num=-9
#     print(soup2)
#     print(last_num)
    
    if pages<2:
        return articles
    
    for p in range(1,pages):
#         new_soup=soup2.replace(soup2[ last_num:-5],str(int(soup2[ last_num:-5])-p))#也可使用序列切片的方式。
        new_soup=soup2.replace(soup2[begin:end],str(int(soup2[begin:end])-p+1))
        print(new_soup)
        #string = 'abcdafg'  newstr = string[:4] + 'e' + string[5:]         [-9:-5]=str(int(soup2[-9:-5])-p)
        response2 = rq.get(new_soup,cookies={'over18': '1'})
        html_doc2 = response2.text
        soups = BeautifulSoup(html_doc2, 'lxml')
        
        for ns in soups.find_all("div", class_= "r-ent"):
            if (ns.find('a')) and (response.status_code == 200):
                date=ns.find("div",class_= "date").string
                date=str(this_year)+'/'+date if int(date[0:2])<=2 else str(this_year-1)+'/'+date
                date=date.replace(" ","")
                name=ns.find('a').string
                push=ns.find("div",class_= "nrec").string
                try:
                    push=100 if push=='爆' else push
                    push=-100 if push=='XX' else push 
                    push=int(push.replace("X","-"))*10 if push.find("X")==0 else push
                except:
                    continue
                author=ns.find("div",class_= "author").string
                url=ns.find("div",class_= "title").a["href"]
                articles.append({'board': board,'title': name,'push': push,'date': date,'author': author,'url': "https://www.ptt.cc"+url}) 
    articles_pd=pd.DataFrame(articles)
    
    return articles

In [2]:
url="https://www.ptt.cc/bbs/sex/index.html"
pages=10
raw_datas=pttcrawler(url,pages)
raw_datas

TypeError: pttcrawler() takes 0 positional arguments but 2 were given

In [336]:
import pymysql
import time
# import charts
# 資料庫參數設定
db_settings = {
    "host": "127.0.0.1",
    "port": 3306,
    "user": "root",
    "password": "As123459362",
    "db": "pttcrawler",
    "charset": "utf8",
    "autocommit":True
}
time_start = time.time() #開始計時

try:
    # 建立Connection物件
    conn = pymysql.connect(**db_settings)
    # 建立Cursor物件
    with conn.cursor() as cursor:
      #資料表相關操作
        command = "INSERT INTO ptt_data(title, board, push_no, date, author, url)VALUES(%s, %s, %s, %s, %s, %s)"
        command_log = "INSERT INTO log(datetime, task, status, record_des, errmsg) VALUES(%s, %s, %s, %s, %s)"
        # 紀錄開始
        cursor.execute(command_log, (datetime.datetime.now(), "pttcrawler", "start", "", ""))
        # 執行
        for raw_data in raw_datas:
            try:
                cursor.execute(command, (raw_data["title"], raw_data["board"], raw_data["push"], raw_data["date"], raw_data["author"], raw_data["url"]))
                conn.commit()
            except Exception as err:
                cursor.execute(command_log, (datetime.datetime.now(), "pttcrawler", "wrong", str(raw_data), str(err)))
                continue
        # 紀錄結束
        cursor.execute(command_log, (datetime.datetime.now(), "pttcrawler", "finish", "", ""))
except Exception as ex:
    print(ex)
        
time_end = time.time()    #結束計時
time_c= time_end - time_start   #執行所花時間
print('time cost', time_c, 's')

time cost 0.09163832664489746 s


In [12]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import re
from flask import Flask
from flask import jsonify, request
import pymysql
import json

app = Flask(__name__)
@app.route("/", methods=['GET'])
def HELLO():
    return "HELLO"

@app.route("/pttcrawler", methods=['POST'])
def pttcrawler():
    # post資料 送在 body 的 raw data裡，可以用 json 轉
    data=request.get_data()
    print(json.loads(data))
    
    if 'url' in request.args:
        url = str(request.args['url'])
    else:
        return "NO_URL",500
    if 'pages' in request.args:
        pages = int(request.args['pages'])
    else:
        pages = 1
    response = rq.get(url,cookies={'over18': '1'})
    html_doc = response.text
    soup = BeautifulSoup(html_doc, 'lxml')
    
    this_year = datetime.datetime.now().year
    
    articles = []
    board=soup.find("a",class_= "board")["href"]
    board = re.search('bbs\/(.*)\/', board)[1]
    wrong=0
    for s in soup.find_all("div", class_= "r-ent"):
        if (s.find('a')) and (response.status_code == 200):
            #requests.get() 的結果是 request.Response 物件, 我們可以先透過該物件的 statu_code 屬性取得 server 回覆的狀態碼
            #(例如 200 表示正常, 404 表示找不到網頁等), 若狀態碼為 200, 代表正常回應
            date=s.find("div",class_= "date").string
            date=str(this_year)+'/'+date if int(date[0:2])<=2 else str(this_year-1)+'/'+date
            date=date.replace(" ","")
            name=s.find('a').string
            push=s.find("div",class_= "nrec").string
            try:
                push=100 if push=='爆' else push
                push=-100 if push=='XX' else push 
                push=int(push.replace("X","-"))*10 if push.find("X")==0 else push
            except:
                continue
            author=s.find("div",class_= "author").string
            url=s.find("div",class_= "title").a["href"]
            articles.append({'board': board,'title': name,'push': push,'date': date,'author': author,'url': "https://www.ptt.cc"+url}) 
            
    soup2="https://www.ptt.cc"+soup.find_all(class_="btn wide")[1]["href"]#第二個([1])class是"btn wide"之下的href值
    #找出這是哪一個版
    begin=soup2.find("index")+5
    end=soup2.find(".html")
    
    if pages<2:
        return  "success, there are "+str(wrong)+" mistakes",200
    
    for p in range(1,pages):
#         new_soup=soup2.replace(soup2[ last_num:-5],str(int(soup2[ last_num:-5])-p))#也可使用序列切片的方式。
        new_soup=soup2.replace(soup2[begin:end],str(int(soup2[begin:end])-p+1))
        #string = 'abcdafg'  newstr = string[:4] + 'e' + string[5:]         [-9:-5]=str(int(soup2[-9:-5])-p)
        response2 = rq.get(new_soup,cookies={'over18': '1'})
        html_doc2 = response2.text
        soups = BeautifulSoup(html_doc2, 'lxml')
        
        for ns in soups.find_all("div", class_= "r-ent"):
            if (ns.find('a')) and (response.status_code == 200):
                date=ns.find("div",class_= "date").string
                date=str(this_year)+'/'+date if int(date[0:2])<=2 else str(this_year-1)+'/'+date
                date=date.replace(" ","")
                name=ns.find('a').string
                push=ns.find("div",class_= "nrec").string
                try:
                    push=100 if push=='爆' else push
                    push=-100 if push=='XX' else push 
                    push=int(push.replace("X","-"))*10 if push.find("X")==0 else push
                except:
                    continue
                author=ns.find("div",class_= "author").string
                url=ns.find("div",class_= "title").a["href"]
                articles.append({'board': board,'title': name,'push': push,'date': date,'author': author,'url': "https://www.ptt.cc"+url}) 
        
        # 資料庫參數設定
    db_settings = {
        "host": "127.0.0.1",
        "port": 3306,
        "user": "root",
        "password": "As123459362",
        "db": "pttcrawler",
        "charset": "utf8",
        "autocommit":True
    }
    print(db_settings)
    try:
        # 建立Connection物件
        conn = pymysql.connect(**db_settings)
        # 建立Cursor物件
        with conn.cursor() as cursor:
          #資料表相關操作
            command = "INSERT INTO ptt_data(title, board, push_no, date, author, url)VALUES(%s, %s, %s, %s, %s, %s)"
            command_log = "INSERT INTO log(datetime, task, status, record_des, errmsg) VALUES(%s, %s, %s, %s, %s)"
            # 紀錄開始
            cursor.execute(command_log, (datetime.datetime.now(), "pttcrawler", "start", "", ""))
            # 執行
            for article in articles:
                try:
                    cursor.execute(command, (article["title"], article["board"], article["push"], article["date"], article["author"], article["url"]))
                    conn.commit()
                except Exception as err:
                    wrong=wrong+1
                    cursor.execute(command_log, (datetime.datetime.now(), "pttcrawler", "wrong", str(article), str(err)))
                    continue
            # 紀錄結束
            cursor.execute(command_log, (datetime.datetime.now(), "pttcrawler", "finish", "", ""))
    except Exception as ex:
        print(ex)

    return  "success, there are "+str(wrong)+" mistakes",200

if __name__ == '__main__':
    app.debug = False
    app.config['JSON_AS_ASCII'] = False
    app.run(host='localhost', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)


b'{"id":"00000000-0000-0000-0000-000000000000","docnoType":"OB","docnoDate":"2021-11-07","docnoSeq":0,"purchasingUnitId":"0011110","nfsStationId":"10126","type":1,"pickupDate":"2021-11-02","total":269,"userId":null,"orderProducts":[{"orderId":"00000000-0000-0000-0000-000000000000","docnoType":"OB","docnoDate":"2021-11-07","docnoSeq":0,"seqNo":0,"productId":"4710713631106","isReplenishment":true,"vendorId":"11457","productType":"\xe5\xa4\xa7\xe5\xae\x97\xe9\xa3\x9f\xe5\x93\x81","productClass":"Y32","productName":"\xe7\xb6\x93\xe5\x85\xb8\xe9\xa6\x99\xe6\xbb\xb7\xe8\xb1\xac\xe6\x8e\x92","unitType":"Quantity","unit":"\xe5\x8c\x85","price":269,"amount":1,"subtotal":269,"analyticsCodeId":"Y3206","spec":"1.6kg.\xe5\x9b\xba1000g"}]}'
*-----*
{'id': '00000000-0000-0000-0000-000000000000', 'docnoType': 'OB', 'docnoDate': '2021-11-07', 'docnoSeq': 0, 'purchasingUnitId': '0011110', 'nfsStationId': '10126', 'type': 1, 'pickupDate': '2021-11-02', 'total': 269, 'userId': None, 'orderProducts': [{'or

127.0.0.1 - - [07/Nov/2021 20:00:28] "POST /pttcrawler?url=https://www.ptt.cc/bbs/sex/index.html&pages=10 HTTP/1.1" 200 -


{'host': '127.0.0.1', 'port': 3306, 'user': 'root', 'password': 'As123459362', 'db': 'pttcrawler', 'charset': 'utf8', 'autocommit': True}
name 'raw_data' is not defined
